Panel ve buton isimlendirmeler şu şekilde gidiyor:
        .tab
        .panel
            .pushbutton
                script.py
                icon.png    
            (diğer tuş tipleri için pyrevit bundle types diye bakılabilir)

Fonksiyona dair genel tanımlar şu şekilde yapılır:

In [ ]:
# -*- coding: utf-8 -*-
__title__ = "ID to Mark"                           # Name of the button displayed in Revit UI
__doc__ = """Version = 1.0
Yazdığınız kodları test etmenizi sağlar"""              # Button Description shown in Revit UI
__author__ = "Baha YALNIZ" 

En genel importlar şu şekilde:

In [ ]:
import os, sys, math, datetime, time                                    # Regular Imports
from Autodesk.Revit.DB import * 
from pyrevit import revit, forms 

clr import edilerek .NET kütüphaneleri python ile kullanılabilir hale getirilir. Python'daki list biçimi her zaman istenildiği gibi olmayabilir. Bu yüzden .NET kütüphanelerinden özel bir list biçimi import edilir:

In [ ]:
import clr                                  # Common Language Runtime. Makes .NET libraries accessinble
clr.AddReference("System")                  # Refference System.dll for import.
from System.Collections.Generic import List # List<ElementType>() <- it's special type of list from .NET framework that RevitAPI requires
# List_example = List[ElementId]()          # use .Add() instead of append or put python list of ElementIds in parentesis.


En temel değişkenler şu şekilde: 

In [ ]:
doc   = __revit__.ActiveUIDocument.Document   # Document   class from RevitAPI that represents project. Used to Create, Delete, Modify and Query elements from the project.
uidoc = __revit__.ActiveUIDocument          # UIDocument class from RevitAPI that represents Revit project opened in the Revit UI.
app   = __revit__.Application                 # Represents the Autodesk Revit Application, providing access to documents, options and other application wide data and settings.
PATH_SCRIPT = os.path.dirname(__file__)     # Absolute path to the folder where script is placed.

active_view  = doc.ActiveView
active_level = doc.ActiveView.GenLevel


Fonksiyon bir "Transaction" içinde çalıştırılır. Bu yapılmazsa, dosyaya müdahale mümkün olmaz.

In [ ]:
if __name__ == '__main__':
    
    # AVOID  placing Transaction inside of your loops! It will drastically reduce perfomance of your script.
    t = Transaction(doc,__title__)  # Transactions are context-like objects that guard any changes made to a Revit model.

    # You need to use t.Start() and t.Commit() to make changes to a Project.
    t.Start()  # <- Transaction Start

    all_floors = FilteredElementCollector(doc).OfCategory(BuiltInCategory.OST_Floors).WhereElementIsNotElementType().ToElements()
    
    n = 0
    for floor in all_floors:
        floor_mark = floor.get_Parameter(BuiltInParameter.ALL_MODEL_MARK)
        floor_mark.Set(str(floor.Id))
 
    t.Commit()  # <- Transaction End

Yeni bir obje yaratılacağı zaman dökümantasyonda bakılması gereken iki konum var:

    1- Oluşturulmak istenen objenin kendi classının içindeki metotlardan "create" metodu aranabilir.
    2- Create metodu olmayan classlar için "Document Class" altındaki metotlardan gerekli create metodu bulunabilir.

In [ ]:
from Autodesk.Revit.DB.Structure import StructuralType

In [ ]:
with Transaction(doc,__title__) as t:
    t.Start()
    #Aşağıdaki kodların hepsi bu aralığa yerleştirilmeli.
    t.commit()

In [ ]:
# ╔╦╗╔═╗═╗ ╦╔╦╗
#  ║ ║╣ ╔╩╦╝ ║
#  ╩ ╚═╝╩ ╚═ ╩ TEXT
# ==================================================
# # ARGUMENTS
text_type_id = FilteredElementCollector(doc).OfClass(TextNoteType).FirstElementId()
pt           = XYZ(0,0,0)
text         = 'Hello BIM World!'
# CREATE TEXT NOTE
TextNote.Create(doc, active_view.Id, pt, text, text_type_id)

In [ ]:
# ╦═╗╔═╗╔═╗╔╦╗
# ╠╦╝║ ║║ ║║║║
# ╩╚═╚═╝╚═╝╩ ╩ ROOM
#==================================================
# ARGUMENTS
pt = UV(10,0)

# CREATE ROOM
room = doc.Create.NewRoom(active_level, pt)

# CREATE ROOM TAG
room_link = LinkElementId(room.Id)
doc.Create.NewRoomTag(room_link, pt, active_view.Id )

In [ ]:
# ╦  ╦╔╗╔╔═╗╔═╗
# ║  ║║║║║╣ ╚═╗
# ╩═╝╩╝╚╝╚═╝╚═╝ DETAIL LINES
# ==================================================
# ARGUMENTS
pt_start = XYZ(20,0,0)
pt_end   = XYZ(20,5,0)
curve    = Line.CreateBound(pt_start, pt_end)

#CREATE DETAIL LINE
detail_line = doc.Create.NewDetailCurve(active_view, curve)

In [ ]:
# ╦ ╦╔═╗╦  ╦  ╔═╗
# ║║║╠═╣║  ║  ╚═╗
# ╚╩╝╩ ╩╩═╝╩═╝╚═╝ WALLS
#==================================================
# ARGUMENTS
pt_start = XYZ(30,0,0)
pt_end   = XYZ(30,5,0)
curve    = Line.CreateBound(pt_start, pt_end)

#CREATE A WALL
wall = Wall.Create(doc, curve, active_level.Id, False)

In [ ]:
# ╦ ╦╦╔╗╔╔╦╗╔═╗╦ ╦╔═╗
# ║║║║║║║ ║║║ ║║║║╚═╗
# ╚╩╝╩╝╚╝═╩╝╚═╝╚╩╝╚═╝ WINDOWS
#==================================================
# ARGUMENTS
host_wall = doc.GetElement(ElementId(779500))
pt_start = XYZ(30,0,0)
pt_end   = XYZ(30,5,0)
pt_mid   = (pt_start + pt_end) /2
window_type = FilteredElementCollector(doc).OfCategory(BuiltInCategory.OST_Windows)\
                .WhereElementIsElementType().FirstElement()

# CREATE A WINDOW
window = doc.Create.NewFamilyInstance(pt_mid, window_type, host_wall, StructuralType.NonStructural )

In [ ]:
# ╔═╗╔═╗╔╦╗╦╦ ╦ ╦  ╦╔╗╔╔═╗╔╦╗╔═╗╔╗╔╔═╗╔═╗
# ╠╣ ╠═╣║║║║║ ╚╦╝  ║║║║╚═╗ ║ ╠═╣║║║║  ║╣
# ╚  ╩ ╩╩ ╩╩╩═╝╩   ╩╝╚╝╚═╝ ╩ ╩ ╩╝╚╝╚═╝╚═╝ FAMILY INSTANCE
#==================================================
# EXTRA FUNCTION
def get_type_by_name(type_name):
    """Extra Function to get Family Type by name."""
    # CREATE RULE
    param_type = ElementId(BuiltInParameter.ALL_MODEL_TYPE_NAME)
    f_param    = ParameterValueProvider(param_type)
    evaluator  = FilterStringEquals()
    f_rule     = FilterStringRule(f_param, evaluator, type_name, True) # Revit 2023 does not need last argument!

    # CREATE FILTER
    filter_type_name = ElementParameterFilter(f_rule)

    # GET ELEMENTS
    return FilteredElementCollector(doc).WherePasses(filter_type_name).WhereElementIsElementType().FirstElement()

# ARGUMENTS
pt = XYZ(40,5,0)
symbol = get_type_by_name('Placeholder - Type B')

# CREATE AN ELEMENT
element = doc.Create.NewFamilyInstance(pt, symbol, StructuralType.NonStructural)

In [ ]:
    # ╔═╗╦ ╦╔═╗╔═╗╔╦╗╔═╗
    # ╚═╗╠═╣║╣ ║╣  ║ ╚═╗
    # ╚═╝╩ ╩╚═╝╚═╝ ╩ ╚═╝ SHEETS
    #==================================================
    # ARGUMENTS
    title_block_id = FilteredElementCollector(doc).OfCategory(BuiltInCategory.OST_TitleBlocks)\
                        .WhereElementIsNotElementType().FirstElementId()
    
    # CREATE SHEET
    new_sheet = ViewSheet.Create(doc, title_block_id)
    new_sheet.SheetNumber = 'Random Sheet Number' #<- Unique !
    new_sheet.Name        = 'Random Name'

In [ ]:
    # ╦  ╦╦╔═╗╦ ╦╔═╗
    # ╚╗╔╝║║╣ ║║║╚═╗
    #  ╚╝ ╩╚═╝╚╩╝╚═╝ VIEWS
    #==================================================
    # ARGUMENTS
    all_view_types = FilteredElementCollector(doc).OfClass(ViewFamilyType).ToElements()
    view_3D_type = [vt for vt in all_view_types if vt.ViewFamily == ViewFamily.ThreeDimensional][0]
    
    # CREATE 3D VIEW
    new_3D = View3D.CreateIsometric(doc, view_3D_type.Id)

In [ ]:

    # ╦═╗╔═╗╔═╗╦╔═╗╔╗╔
    # ╠╦╝║╣ ║ ╦║║ ║║║║
    # ╩╚═╚═╝╚═╝╩╚═╝╝╚╝ REGION
    #==================================================
    # ARGUMENTS
    region_type_id = doc.GetDefaultElementTypeId(ElementTypeGroup.FilledRegionType)
    
    # POINTS
    pt_0 = XYZ(50, 0, 0)
    pt_1 = XYZ(55, 0, 0)
    pt_2 = XYZ(55, 5, 0)
    pt_3 = XYZ(50, 5, 0)
    
    # LINES
    l_0 = Line.CreateBound(pt_0, pt_1)
    l_1 = Line.CreateBound(pt_1, pt_2)
    l_2 = Line.CreateBound(pt_2, pt_3)
    l_3 = Line.CreateBound(pt_3, pt_0)
    
    # BOUNDARY
    boundary = CurveLoop()
    boundary.Append(l_0)
    boundary.Append(l_1)
    boundary.Append(l_2)
    boundary.Append(l_3)
    
    # LIST OF BOUNDARIES
    list_boundaries = List[CurveLoop]()
    list_boundaries.Add(boundary)
    
    # CREATE FILLED REGION
    region = FilledRegion.Create(doc, region_type_id, active_view.Id, list_boundaries)

In [ ]:
    # ╔═╗╦  ╔═╗╔═╗╦═╗
    # ╠╣ ║  ║ ║║ ║╠╦╝
    # ╚  ╩═╝╚═╝╚═╝╩╚═ FLOOR
    #==================================================
    # ARGUMENTS
    floor_type_id = doc.GetDefaultElementTypeId(ElementTypeGroup.FloorType)
    floor_type    = doc.GetElement(floor_type_id)
    
    # POINTS
    pt_0 = XYZ(60, 0, 0)
    pt_1 = XYZ(65, 0, 0)
    pt_2 = XYZ(65, 5, 0)
    pt_3 = XYZ(60, 5, 0)
    
    # LINES
    l_0 = Line.CreateBound(pt_0, pt_1)
    l_1 = Line.CreateBound(pt_1, pt_2)
    l_2 = Line.CreateBound(pt_2, pt_3)
    l_3 = Line.CreateBound(pt_3, pt_0)
    
    # BOUNDARY
    boundary = CurveArray()
    boundary.Append(l_0)
    boundary.Append(l_1)
    boundary.Append(l_2)
    boundary.Append(l_3)
    
    # CREATE FLOOR
    new_floor = doc.Create.NewFloor(boundary, floor_type, active_level, False)

In [ ]:
    # ╔═╗╔═╗╔═╗╦ ╦  ╔═╗╦  ╔═╗╔╦╗╔═╗╔╗╔╔╦╗╔═╗
    # ║  ║ ║╠═╝╚╦╝  ║╣ ║  ║╣ ║║║║╣ ║║║ ║ ╚═╗
    # ╚═╝╚═╝╩   ╩   ╚═╝╩═╝╚═╝╩ ╩╚═╝╝╚╝ ╩ ╚═╝ COPY ELEMENTS
    # ==================================================
    # # GET ELEMENTS TO COPY
    all_floors_in_view = FilteredElementCollector(doc, active_view.Id).OfCategory(BuiltInCategory.OST_Floors).WhereElementIsNotElementType().ToElementIds()
    elements_to_copy = List[ElementId](all_floors_in_view)
    
    # COPY ELEMENTS (Multiple Times)
    for i in range(1,6):
        vector = XYZ(2*i, 10*i, 0)
        ElementTransformUtils.CopyElements(doc, elements_to_copy, vector)

In [ ]:
    # ╔╦╗╔═╗╦  ╔═╗╔╦╗╔═╗  ╔═╗╦  ╔═╗╔╦╗╔═╗╔╗╔╔╦╗╔═╗
    #  ║║║╣ ║  ║╣  ║ ║╣   ║╣ ║  ║╣ ║║║║╣ ║║║ ║ ╚═╗
    # ═╩╝╚═╝╩═╝╚═╝ ╩ ╚═╝  ╚═╝╩═╝╚═╝╩ ╩╚═╝╝╚╝ ╩ ╚═╝ DELETE ELEMENTS
    # ==================================================
    # # GET ELEMENTS TO COPY
    all_floors_in_view = FilteredElementCollector(doc, active_view.Id).OfCategory(BuiltInCategory.OST_Floors).WhereElementIsNotElementType().ToElementIds()
    element_to_delete  = List[ElementId](all_floors_in_view)
    
    # DELETE ELEMENTS
    doc.Delete(element_to_delete)

-----